In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.0.2


In [2]:
import pickle
import pandas as pd
import numpy as np

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [4]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

## Q1

In [5]:
year = 2021
month = 2

df = read_data(f'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year:04d}-{month:02d}.parquet')

In [6]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [7]:
print(np.mean(y_pred))

16.191691679979066


## Q1 Answer
### Mean predicted duration is 16.19

## Q2

In [8]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

def save_results(df, y_pred, output_file):
    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']
    df_result['predicted_duration'] = y_pred

    df_result.to_parquet(
        output_file,
        engine='pyarrow',
        compression=None,
        index=False
    )

In [9]:
!mkdir output

mkdir: cannot create directory ‘output’: File exists


In [10]:
output_file = f'./output/fhv_tripdata{year:04d}-{month:02d}.parquet'
save_results(df, y_pred, output_file)

In [11]:
! ls -lh output/

total 65M
-rwxrwxrwx 1 andre andre 19M Jun 27 16:18 fhv_tripdata2021-02.parquet
-rwxrwxrwx 1 andre andre 46M Jun 27 16:16 fhv_tripdata2021-03.parquet


## Q2 Answer
### Output file size is 19M

## Q3

In [12]:
# !jupyter nbconvert --to script starter.ipynb
## Commented out to not clobber finished script on subsequent runs

## Q3 Answer
### `jupyter nbconvert --to script starter.ipynb`

## Q4

In [13]:
!pipenv install scikit-learn==1.0.2 pandas numpy pyarrow fastparquet

Installing scikit-learn==1.0.2...
Adding scikit-learn to Pipfile's [packages]...
✔ Installation Succeeded 
Installing pandas...
Adding pandas to Pipfile's [packages]...
✔ Installation Succeeded 
Installing numpy...
Adding numpy to Pipfile's [packages]...
✔ Installation Succeeded 
Installing pyarrow...
Adding pyarrow to Pipfile's [packages]...
✔ Installation Succeeded 
Installing fastparquet...
Adding fastparquet to Pipfile's [packages]...
✔ Installation Succeeded 
Installing dependencies from Pipfile.lock (174e10)...
  🐍   ▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉ 0/0 — 00:00:00
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.


In [14]:
!cat Pipfile.lock | grep scikit-learn -A 2

        "scikit-learn": {
            "hashes": [
                "sha256:08ef968f6b72033c16c479c966bf37ccd49b06ea91b765e1cc27afefe723920b",


## Q4 Answer
### Hash is "sha256:08ef968f6b72033c16c479c966bf37ccd49b06ea91b765e1cc27afefe723920b"

## Q5

In [15]:
!python starter.py 2021 3

Mean predicted duration is 16.298821614015107


## Q5 Answer
### Mean predicted duration is 16.298821614015107"

## Q6

In [16]:
!cat Dockerfile

FROM agrigorev/zoomcamp-model:mlops-3.9.7-slim

# do stuff here
RUN pip install -U pip
RUN pip install pipenv 

COPY [ "Pipfile", "Pipfile.lock", "./" ]

RUN pipenv install --system --deploy

COPY [ "starter.py", "./" ]

RUN mkdir output

CMD [ "python", "starter.py", "2021", "4" ]

## Q6 answer
### Mean predicted duration is 9.967573179784523
I assume the Docker file will be run to verify this